In [1]:
import nltk
from nltk.corpus import wordnet as wn
import json
import polars as pl

# Download WordNet data
nltk.download('wordnet')

import sys
sys.path.append('..')
from utils import utils as u

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rossellaborra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/rossellaborra/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rossellaborra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def compute_similarity(definition_1_tokens, definition_2_tokens):

    if len(definition_2_tokens) == 0 or len(definition_1_tokens) == 0:
        return 0

    min_len = 0

    if len(definition_1_tokens) > len(definition_2_tokens):
        min_len = len(definition_2_tokens)
    else:
        min_len = len(definition_1_tokens)

    return (len(set(definition_1_tokens).intersection(set(definition_2_tokens))) / min_len)


In [3]:
# read csv with polars module
df = json.loads(pl.read_csv("../datasets/TLN-definitions-23.tsv", separator='\t').write_json())

data = {}

In [4]:
for col in df['columns']:
    if col['name'] != '1':
        data[col['name']] = {}
        count = 1
        for v in col['values']:
            data[col['name']][count] = u.noise_reduction_en(v)
            count += 1


In [5]:
def get_all_synsets_definitions(word):
    synset_list = wn.synsets(word)
    definitions = {}
    for s in synset_list:
        definitions[s] = u.noise_reduction_en(s.definition())
    return definitions

devo vedere tra le definizioni di tutti i synset della definizione tokenizzata quella più simile alla definizione di partenza e prendere il synset

In [6]:
def get_all_synsets(tagged_token_list):
    tokens_definitions_list = []

    for token in tagged_token_list:
        tokens_definitions_list.append(get_all_synsets_definitions(token))
    return tokens_definitions_list


In [7]:
def get_best_synset(tagged_tokens_list, list_of_tagged_tokens_list):
    best_synsets = {}
    max_score = 0
    for synset_list in list_of_tagged_tokens_list:
        for synset in synset_list.keys():
            score = compute_similarity(tagged_tokens_list, synset_list[synset])
            if score > 0.7 and score >= max_score:
                best_synsets[synset] = score
                max_score = score
    return best_synsets


In [8]:
def compute_similarity(definition_1_tokens, definition_2_tokens):

    min_len = 0

    if len(definition_1_tokens) > len(definition_2_tokens):
        min_len = len(definition_2_tokens)
    else:
        min_len = len(definition_1_tokens)

    return (len(set(definition_1_tokens).intersection(set(definition_2_tokens))) / min_len)


In [9]:
result = {}

for word in data:
    result[word] = {}
    for id in data[word]:
        best_synset = get_best_synset(
            data[word][id], get_all_synsets(data[word][id]))

        if len(best_synset) > 0:
            result[word][id] = best_synset

result


{'door': {3: {Synset('door.n.05'): 0.75},
  6: {Synset('obstruct.v.02'): 1.0},
  7: {Synset('unlock.v.01'): 1.0},
  12: {Synset('outside.r.02'): 1.0},
  13: {Synset('inside.r.02'): 1.0, Synset('outside.r.02'): 1.0},
  19: {Synset('obstruct.v.02'): 1.0},
  30: {Synset('space.n.07'): 1.0}},
 'ladybug': {4: {Synset('back.v.06'): 1.0},
  7: {Synset('back.v.06'): 1.0},
  16: {Synset('back.v.06'): 1.0},
  26: {Synset('back.v.06'): 1.0},
  27: {Synset('back.v.06'): 1.0}},
 'pain': {2: {Synset('hurt.v.05'): 1.0},
  9: {Synset('express.r.01'): 1.0},
  12: {Synset('result.n.03'): 1.0},
  14: {Synset('harm.v.01'): 1.0},
  17: {Synset('harm.v.01'): 1.0},
  19: {Synset('write_out.v.02'): 1.0}},
 'blurriness': {18: {Synset('function.n.02'): 1.0}}}

In [10]:
for word in result:
    
    for synset in wn.synsets(word):
        for id in result[word].keys():
            if synset in result[word][id]:
                print(word, 'ok')
                break

door ok


- considerare gli esempi dei synset delle parole della definizione e prendere quelle più ricorrenti  (magari anche dei loro ipo e ipernimi??)
- considerare le definizioni dei vari synset delle parole della definizione e prendere i synset delle definizioni più simili
- usare sto principio "genus-differentia"
- prendo dalle definizioni le parole più ricorrenti, e poi dalle definizioni dei loro synset provo a ricavarmi la parola target
- mi prendo da tutte le definizioni a mia disposizione le parole importnti e faccio un corpus importante, poi cerco tra tutte le definizioni in wordnet quella con più parole in comune

In [11]:
# creo la struttura dati dalle definizioni per ogni parola
for col in df['columns']:
    if col['name'] != '1':
        data[col['name']] = {}
        token_list = []
        for v in col['values']:
            token_list += u.noise_reduction_en(v)

        dict_token_freq = {}

        for token in token_list:
            if token not in dict_token_freq:
                dict_token_freq[token] = 1
            else:
                dict_token_freq[token] += 1

        data[col['name']] = dict(sorted(dict_token_freq.items(), key=lambda item: item[1], reverse=True))




In [12]:
for word in data:
    for token in data[word]:
        if data[word][token] > 5:
            synsets = wn.synsets(token)
            hypernims = []
            hyponims = []
            for syns in synsets:
                hypernims += syns.hypernyms()
                hyponims += syns.hyponyms()
            print(hypernims)
        else:
            break


[Synset('area.n.05'), Synset('position.n.07'), Synset('opportunity.n.01'), Synset('gathering.n.01'), Synset('populate.v.01')]
[Synset('physical_entity.n.01'), Synset('goal.n.01'), Synset('constituent.n.04'), Synset('content.n.05'), Synset('computer_science.n.01'), Synset('disapprove.v.02'), Synset('be.v.01')]
[Synset('right.n.01'), Synset('right.n.01'), Synset('way.n.06'), Synset('code.n.02'), Synset('operation.n.04'), Synset('approach.n.02'), Synset('recover.v.01'), Synset('reach.v.01')]
[Synset('area.n.01'), Synset('outside.n.01'), Synset('tournament.n.01'), Synset('public_knowledge.n.01'), Synset('change_state.v.01'), Synset('start.v.06'), Synset('undo.v.01'), Synset('yield.v.01'), Synset('arise.v.04'), Synset('move.v.15'), Synset('expose.v.03')]
[Synset('accept.v.03'), Synset('give.v.03'), Synset('assign.v.02'), Synset('yield.v.01'), Synset('calculate.v.02'), Synset('discount.v.02'), Synset('permit.v.01')]
[Synset('end.n.02'), Synset('section.n.01'), Synset('finish.n.09'), Synset('

In [13]:
import nltk
from nltk.corpus import wordnet

# Scarica i dati di WordNet (se necessario)
nltk.download('wordnet')

# Funzione per ottenere il Synset corrispondente a una parola chiave
def get_synset_from_word(keyword):
    synsets = wordnet.synsets(keyword)
    if synsets:
        return synsets[0]  # Ritorna il primo Synset trovato
    else:
        return None

# Esempio di definizioni nel corpus (puoi aggiungere le tue definizioni qui)
definitions_corpus = [
    "Il gatto è un mammifero carnivoro della famiglia dei Felidi.",
    "Il cane è un animale domestico appartenente alla famiglia dei Canidi.",
    "La rosa è un fiore appartenente al genere Rosa.",
    "La mela è il frutto del melo, pianta della famiglia delle Rosacee."
]

# Estrai le parole chiave dalle definizioni (in questo caso, i sostantivi)
keywords = []
for definition in definitions_corpus:
    tokens = nltk.word_tokenize(definition)
    tagged_tokens = nltk.pos_tag(tokens)
    for word, pos in tagged_tokens:
        if pos.startswith('NN'):  # Filtra solo i sostantivi (NN, NNS, NNP, NNPS)
            keywords.append(word.lower())

# Cerca i Synset corrispondenti alle parole chiave
synsets_found = {}
for keyword in keywords:
    synset = get_synset_from_word(keyword)
    if synset:
        synsets_found[keyword] = synset

# Stampa i risultati
for keyword, synset in synsets_found.items():
    print(f"Parola chiave: {keyword}")
    print(f"Definizione: {synset.definition()}")
    print(f"Esempi: {synset.examples()}")
    print("---")


Parola chiave: il
Definizione: a midwestern state in north-central United States
Esempi: []
---
Parola chiave: un
Definizione: an organization of independent states formed in 1945 to promote international peace and security
Esempi: []
---
Parola chiave: cane
Definizione: a stick that people can lean on to help them walk
Esempi: []
---
Parola chiave: la
Definizione: a white soft metallic element that tarnishes readily; occurs in rare earth minerals and is usually classified as a rare earth
Esempi: []
---
Parola chiave: al
Definizione: a silvery ductile metallic element found primarily in bauxite
Esempi: []
---
Parola chiave: rosa
Definizione: large genus of erect or climbing prickly shrubs including roses
Esempi: []
---


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rossellaborra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
